In [39]:
# initial setup
# %run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Limpieza de datos. Apply. Expresiones regulares.

La limpieza es un paso necesario en todo proyecto de datos. 

Podemos resumir el proceso de limpieza de datos en las siguientes cinco tareas:

**1. Resolver problemas de formato y asignar tipos de datos correctos.**

Por ejemplo, cuando al pasar de CSV a Pandas una fecha no se importa correctamente como puede ser el caso de un campo fecha donde se importa 20090609231247 en lugar de 2009-06-09 23:12:47.

El formato en que se encuentran los datos determina qué operaciones pueden realizarse sobre ellos.

**2. Estandarizar categorías.**

Cuando los datos se recolectaron con un sisstema que no tiene valores tipificadaos, valores que representan la misma categoría pueden estar expresados de forma distinta. Por ejemplo: Arg, AR, Argentina

**3. Corregir valores erróneos.**

Por ejemplo: un valor numérico o inválido para describir el género; o una edad representada por un número negativo o mucho mayor que 100.

**4. Completar datos faltantes.**

Los datasets del mundo real suelen venir con datos faltantes que responden a información que se perdió o nunca se recolectó. Existen varias técnicas para completar datos faltantes. Al proceso de completar datos faltantes se lo llama "imputación".

**5. Organizar el dataset.**

Es importante estructurar las filas y columnas de la forma más conveniente. Para hacerlo se pueden aplicar las reglas del "tidy data".



## Dataset

El Met (Metropolitan Museum of Art) provee datasets de información de más de 420.000 piezas de arte en su colección.

https://github.com/metmuseum/openaccess/

Los problemas que presentan este dataset incluyen:
* Valores faltantes
* Posibles duplicaciones
* Campos con mezcla de campos de tipo numérico con cadenas de caracteres

En esta clase vamos a detectar algunos de los qué campos presentan problemas, y solucionarlos usando las herramientas que adquirimos en las prácticas guiadas.


## Ejercicio 1:

Vamos a leer en la variable `data` los datos del archivo /M2/CLASE_05_Limpieza_de_datos/Data/MetObjects_sample.csv en un `DataFrame` de pandas con el método `read_csv` 

Veamos cuántas filas y columnas tiene el DataFrame data

In [40]:
import pandas as pd

data_location = "../Data/MetObjects_sample.csv"

data = pd.read_csv(data_location, low_memory=False)

print("cantidad de filas: " + str(data.shape[0]))
print("cantidad de columnas: " + str(data.shape[1]))
data.sample(3)

cantidad de filas: 4743
cantidad de columnas: 52


,Unnamed: 0,Object Number,Is Highlight,Is Public Domain,Is Timeline Work,Object ID,Department,AccessionYear,Object Name,Title,...,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL
4368,22300,1977.162.3,False,False,False,27264,Arms and Armor,1977,Sword (Kaskara),Sword (Kaskara),...,NaN,NaN,Swords,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN
1855,53072,22.141.4,False,False,True,61398,Asian Art,1922,Mirror,銅製人物殿閣文花形鏡 高麗|Mirror,...,NaN,NaN,Mirrors,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Mirrors,http://vocab.getty.edu/page/aat/300037682
2791,318560,55.61.64,False,False,True,471629,The Cloisters,1955,Candlestick,Candlestick,...,NaN,NaN,Metalwork-Iron,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",Candlesticks,http://vocab.getty.edu/page/aat/300037588


## Ejercicio 2: Formato y tipos de datos

Vamos a ver de qué tipo de datos es cada columna del DataFrame, y vamos a convertir o dar formato **a alguna de las columnas** que tienen tipo de datos incorrecto.

### 2.a Detectar las columnas que tienen tipo de datos incorrecto

In [41]:
data.dtypes

Unnamed: 0                   int64
Object Number               object
Is Highlight                  bool
Is Public Domain              bool
Is Timeline Work              bool
Object ID                    int64
Department                  object
AccessionYear               object
Object Name                 object
Title                       object
Culture                     object
Period                      object
Dynasty                     object
Reign                       object
Portfolio                   object
Artist Role                 object
Artist Prefix               object
Artist Display Name         object
Artist Display Bio          object
Artist Suffix               object
Artist Alpha Sort           object
Artist Nationality          object
Artist Begin Date           object
Artist End Date             object
Artist Gender               object
Artist ULAN URL             object
Artist Wikidata URL         object
Object Date                 object
Object Begin Date   

### 2.b AccessionYear

Analizar la columna AccessionYear que fue leída como object, y debería ser int.

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos extraer el dato año de los valores no numéricos, y crear una nueva columna en el DataFrame de tipo int que se llame AccessionYearClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

Observación: si la columna tiene valores NaN no vamos a poder convertirla al tipo int. Una opción es reemplazar los valores NaN por algún entero antes de convertir (`fillna`). La otra opción es dejar la columna como tipo float (el tipo de NaN es float).


In [42]:
data.AccessionYear.value_counts()

1963          220
1917          140
1874          126
1929           99
2011           98
             ... 
2005-02-15      1
2020-03-23      1
1986-12-29      1
1873            1
1892            1
Name: AccessionYear, Length: 142, dtype: int64

In [43]:
import re
import numpy as np 

pattern_fecha = "(?P<year>\d{4})(?P<month_day>\-\d\d\-\d\d)*"
pattern_fecha_regex =  re.compile(pattern_fecha)
resultado_fechas = data.AccessionYear.apply(lambda x: x if x is np.NaN else pattern_fecha_regex.match(x))

year_match = resultado_fechas.apply(lambda x: 0 if x is np.NaN else x.group("year"))

data["AccessionYearClean"] = year_match
data["AccessionYearClean"]

0          0
1          0
2          0
3          0
4          0
        ... 
4738    1941
4739    1979
4740    1979
4741    1953
4742    1963
Name: AccessionYearClean, Length: 4743, dtype: object

## Ejercicio 3: Categorias - Valores erróneos

Miremos ahora el campo "Artist Gender"

¿Qué valores toma ese campo? ¿Cómo se distribuyen esos valores? ¿Hay valores nulos?

Queremos definir como categorías válidas Male, Female y Unknown

Y crear una nueva columna en el DataFrame que se llame ArtistGenderClean y tenga estos valores.

Para eso vamos a usar expresiones regulares, apply y lambda.

¿Podemos deducir cómo está representada la categoría Male en el dataset original?

Nota: La propuesta que hacemos para limpiar este campo no es del todo correcta, y vamos a ver por qué más adelante. Pero sirve como ejercicio.


In [47]:
data["ArtistGenderClean"] = data["Artist Gender"].fillna('Unknown')
data["ArtistGenderClean"].value_counts()

Unknown                     3751
|                            583
||                           170
Female                        81
Female|                       43
|||                           41
|Female                       23
||||                          20
||||||                         7
|||||                          3
Female||                       3
|||||||                        3
|Female|                       2
Female||Female                 2
|||||||||||                    2
|||||||||||||||Female|||       1
|||Female|||||                 1
||Female|                      1
|Female||                      1
||Female                       1
Female|Female                  1
|Female|Female|||Female        1
|||||||||||||||||              1
||||||||                       1
Name: ArtistGenderClean, dtype: int64

In [52]:
pattern_pipes = "\|+"
pattern_pipes_regex = re.compile(pattern_pipes)
cadena_reemplazo = ""
data["ArtistGenderClean"] = data["ArtistGenderClean"].apply(lambda x: pattern_pipes_regex.sub(cadena_reemplazo, x))
data["ArtistGenderClean"].value_counts()

Unknown    3751
Male        831
Female      161
Name: ArtistGenderClean, dtype: int64

In [49]:
pattern_female = "(Female)+"
pattern_female_regex = re.compile(pattern_female)
cadena_reemplazo = "Female"
data["ArtistGenderClean"] = data["ArtistGenderClean"].apply(lambda x: pattern_female_regex.sub(cadena_reemplazo, x))
data["ArtistGenderClean"].value_counts()

Unknown    3751
            831
Female      161
Name: ArtistGenderClean, dtype: int64

In [50]:
empty_mask = data["ArtistGenderClean"] == ""
data.loc[empty_mask, "ArtistGenderClean"] = "Male"
data["ArtistGenderClean"].value_counts()

Unknown    3751
Male        831
Female      161
Name: ArtistGenderClean, dtype: int64

## Ejercicio 4: Imputación

Vamos a analizar ahora los campos "Object Date", "Object Begin Date", "Object End Date"

**4.a ¿Cuántos valores nulos hay en "Object Date"? ¿Cuántos en "Object Begin Date"? ¿Cuántos en "Object End Date"?**


In [54]:
null_object_date = data["Object Date"].isnull().sum()
null_object_begin_date = data["Object Begin Date"].isnull().sum()
null_object_end_date = data["Object End Date"].isnull().sum()
print(f"object date: {null_object_date} object begin date: {null_object_begin_date} object end date: {null_object_end_date}")

object date: 1570 object begin date: 0 object end date: 0


**4.b Usaremos los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

1) Vamos a crear una columna nueva ("Object Date 4b") donde copiamos todos los datos de Object Date (para no cambiar los valores originales y nos sirvan para el próximo ejercicio)

2) Vamos a rellenar la columna "Object Date 4b" con la estrategia que planteamos.

In [63]:
data["Object Date 4b"] = data["Object Date"]
mask_object_date_null = data["Object Date"].isnull()
data.loc[mask_object_date_null, "Object Date 4b"] = data.loc[mask_object_date_null, "Object End Date"]
data.loc[mask_object_date_null, ["Object Date 4b", "Object Begin Date", "Object End Date"]]


,Object Date 4b,Object Begin Date,Object End Date
51,0,0,0
103,330,-7000,330
122,0,0,0
136,0,0,0
159,0,0,0
...,...,...,...
2498,1450,1440,1450
2499,-30,-7000,-30
2500,1911,1644,1911
2501,1536,1526,1536


**4.c Usaremos aleatoriamente los valores de "Object Begin Date" o "Object End Date" para imputar los valores de "Object Date" con alguno de esos dos campos.**

Para eso definimos una función get_fill_value que recibe como parámetro una fila da data, y si el valor del campo "Object Date" es nulo devuelve aleatoriamente el valor del campo "Object Begin Date" u "Object End Date" de ese registro.

(En este ejercicio practicamos todo!)

Vamos a contar ahora cuántos registros rellenó con los valores de Object Begin Date y cuántos con Object End Date

## Nota: Organizar el dataset

Para llevar a cabo esta tarea necesitamos algunas herramientas que veremos en la segunda parte de Pandas.

En esa clase vamos a volver a este ejercicio y resolver este punto.

Los campos que vamos a analizar son "Artist Nationality" y "Artist Display Name"

In [ ]:
data["Artist Nationality"].value_counts()

In [ ]:
data["Artist Display Name"].value_counts() 